<a href="https://colab.research.google.com/github/silverstar0727/study-/blob/master/%EC%98%88%EC%A0%9C%EB%A1%9C_%EB%B0%B0%EC%9A%B0%EB%8A%94_%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## numpy를 이용한 신경망 구성

In [16]:
import numpy as np

# n: 배치 사이즈, d_in: 입력 차원, h: 은닉층 차원, d_out: 출력 차원
n, d_in, h, d_out = 64, 1000, 100, 10

# 무작위 입력과 출력데이터 생성
x = np.random.randn(n, d_in)
y = np.random.randn(n, d_out)

# 무작위로 가중치를 조기화
w1 = np.random.randn(d_in, h)
w2 = np.random.randn(h, d_out)

learning_rate = 1e-6
for t in range(500): # epochs 횟수
  
  # forward propagation
  h = x.dot(w1) # 가중치 x 입력
  h_relu = np.maximum(h, 0) # relu와 동일(앞선 계산과 0중 큰것을 출력)
  y_pred = h_relu.dot(w2) # 앞선 계산 x 두번째 가중치 = 예측값

  # loss를 계산하고 출력
  loss = np.square(y_pred - y).sum() # SSE
  if t % 100 == 99:
    print(t, loss)

  # 손실에 대한 w1, w2의 변화도를 계산하고 역전파
  grad_y_pred = 2 * (y_pred - y) # 오차 x 2
  grad_w2 = h_relu.T.dot(grad_y_pred) # h_relu의 전치행렬 x (오차 x 2)
  grad_h_relu = grad_y_pred.dot(w2.T) # 두번째 가중치의 전치 x (h_relu의 전치행렬 x (오차 x 2))
  grad_h = grad_h_relu.copy()
  grad_h[h < 0] = 0
  grad_w1 = x.T.dot(grad_h)

  # 가중치의 갱신
  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2

99 560.7565497684964
199 4.832373132989208
299 0.07309178353451654
399 0.0012391873932189393
499 2.176221215795321e-05


## tensor를 이용한 구현

In [17]:
import torch

dtype = torch.float # 모든 데이터 타입을 실수로 통일
device = torch.device('cuda:0') # gpu에서 실행

# 위와동일한 설정
n, d_in, h, d_out = 64, 1000, 100, 10

# 데이터생성
x = torch.randn(n, d_in, device = device, dtype = dtype)
y = torch.randn(n, d_out, device = device, dtype = dtype)

# 가중치 초기화
w1 = torch.randn(d_in, h, device = device, dtype = dtype)
w2 = torch.randn(h, d_out, device = device, dtype = dtype)

learning_rate = 1e-6
for t in range(500):
  # forward
  h = x.mm(w1)
  h_relu = h.clamp(min = 0)
  y_pred = h_relu.mm(w2)

  # loss계산과 출력
  loss = (y_pred - y).pow(2).sum().item() # item을 안하면, 자료형이 통째로 출력됨
  if t % 100 == 99:
    print(t, loss)

  # 손실에 따른 w1, w2의 변화를 계산 후 역전파
  grad_y_pred = 2 * (y_pred - y)
  grad_w2 = h_relu.t().mm(grad_y_pred)
  grad_h_relu = grad_y_pred.mm(w2.t())
  grad_h = grad_h_relu.clone()
  grad_h[h < 0] = 0
  grad_w1 = x.t().mm(grad_h)

  # gradient descent
  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2

99 378.85345458984375
199 1.3252729177474976
299 0.008014790713787079
399 0.00018893394735641778
499 3.290490712970495e-05


## Autograd 활용

In [18]:
import torch

dtype = torch.float # 모든 데이터 타입을 실수로 통일
device = torch.device('cuda:0') # gpu에서 실행

# 위와동일한 설정
n, d_in, h, d_out = 64, 1000, 100, 10

# 데이터 생성
x = torch.randn(n, d_in, device = device, requires_grad = True)
y = torch.randn(n, d_out, device = device, requires_grad = True)

w1 = torch.randn(d_in, h, device = device, requires_grad = True)
w2 = torch.randn(h, d_out, device = device, requires_grad = True)

learning_rate = 1e-6
for t in range(500):
  # forward propagation
  # tensor만을 사용한 위의 방법과 동일하나, 역전파가 자동으로 되므로 굳이 참조를 갖고 있을 필요가 없음
  y_pred = x.mm(w1).clamp(min = 0).mm(w2)

  # loss
  loss = (y_pred - y).pow(2).sum()
  if t % 100 == 99:
    print(t, loss.item())

  # 역전파
  # requires_grad = True를 갖는 모든 텐서에대함 변화도를 계산함
  # w1, w2는 각각의 gradient에 해당하는 tensor로 변함
  loss.backward()

  # 경사하강법을 이용한 가중치의 수동 갱신
  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad

    # 수동으로 0으로 만들어 줌
    w1.grad.zero_()
    w2.grad.zero_()

99 320.86224365234375
199 1.209773302078247
299 0.008104282431304455
399 0.0002097644319292158
499 3.425555769354105e-05


## nn Module활용



In [19]:
# nn을 활용한 2계층 신경망
import torch

n, d_in, h, d_out = 64, 1000, 100, 10
x = torch.randn(n, d_in)
y = torch.randn(n, d_out)

model = torch.nn.Sequential(
    torch.nn.Linear(d_in, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, d_out),
)

loss_fn = torch.nn.MSELoss(reduction = 'sum')

learning_rate = 1e-4
for t in range(500):
  y_pred = model(x)

  loss = loss_fn(y_pred, y)
  if t % 100 == 99:
    print(t, loss.item())

  model.zero_grad() # 변화도 0으로 만들기
  loss.backward() # 역전파

  with torch.no_grad():
    for param in model.parameters():
      param -= learning_rate * param.grad

99 1.4697588682174683
199 0.0381244458258152
299 0.0023083710111677647
399 0.00017116892558988184
499 1.4096542145125568e-05


## optim 패키지 활용

In [20]:
import torch

n, d_in, h, d_out = 64, 1000, 100, 10
x = torch.randn(n, d_in)
y = torch.randn(n, d_out)

model = torch.nn.Sequential(
    torch.nn.Linear(d_in, h),
    torch.nn.ReLU(),
    torch.nn.Linear(h, d_out),
)
loss_fn = torch.nn.MSELoss(reduction = 'sum')

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for t in range(500):
  y_pred = model(x)
  loss = loss_fn(y_pred, y)
  if t % 100 == 99:
    print(t, loss.item())

  optimizer.zero_grad() # 0으로 초기화
  loss.backward() # 오차 역전파
  optimizer.step() # step() 함수를 호출하면 매개변수가 갱신됨

99 39.763118743896484
199 0.36284488439559937
299 0.000566558912396431
399 1.3267191434351844e-07
499 7.449696415307017e-11


## 사용자 정의 nn.Module

In [26]:
# 2계층 신경망을 직접 정의한 nn.Module 서브클래스를 구현
import torch

class Net(torch.nn.Module):
  def __init__(self, d_in, h, d_out):
    # 2개의 nn.linear모듈을 생성하고 멤버 변수로 지정
    super(Net, self).__init__()
    self.linear1 = torch.nn.Linear(d_in, h)
    self.linear2 = torch.nn.Linear(h, d_out)

  def forward(self, x):
    h_relu = self.linear1(x).clamp(min = 0)
    y_pred = self.linear2(h_relu)

    return y_pred

n, d_in, h, d_out = 64, 1000, 100, 10
x = torch.randn(n, d_in)
y = torch.randn(n, d_out)

model = Net(d_in, h, d_out)
criterion = torch.nn.MSELoss(reduction = 'sum')
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-4)
for t in range(500):
  y_pred = model(x)
  
  loss = criterion(y_pred, y)
  if t % 100 == 99:
    print(y, loss.item())

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

tensor([[ 3.3274e-01,  4.4785e-01, -2.3403e-02,  6.5466e-01, -1.4219e+00,
         -1.2237e-01,  7.4087e-01, -1.4286e+00, -5.9989e-01, -6.7965e-02],
        [ 1.9023e+00,  2.1510e+00, -1.5609e+00, -8.0375e-01,  3.0076e-02,
          2.6737e-02,  1.0366e+00, -5.7379e-01, -1.2106e+00,  9.5888e-01],
        [ 2.3286e-01, -5.5207e-02,  8.3680e-01, -1.7994e-01, -4.3342e-01,
         -1.5770e+00, -3.8999e-01, -1.0351e+00, -1.3367e+00,  1.1195e-01],
        [-6.4231e-02, -4.2564e-02,  2.9434e+00, -2.6326e+00,  4.5489e-01,
          1.7598e+00, -4.7026e-01, -8.3716e-01, -1.5282e+00, -2.3325e-01],
        [-6.8649e-01, -4.8424e-01, -1.6436e+00, -1.1789e+00, -1.2996e+00,
         -4.7453e-01,  1.0192e+00, -3.5635e-01, -4.8131e-01,  6.9148e-01],
        [-7.5340e-01,  8.1665e-01, -1.1109e+00,  1.7986e+00, -1.3695e+00,
         -8.6237e-01, -2.4488e-01,  8.6329e-01, -4.5364e-02,  2.0392e+00],
        [-3.0046e-01,  6.0751e-01,  3.6432e-01, -9.9041e-03,  3.7484e-01,
         -1.0180e+00, -1.2174e+0